In [16]:
#!/usr/bin/python3
"""Testing On Segmentation Task."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
import sys
import math
import h5py
import argparse
import importlib
import data_utils
import numpy as np
import tensorflow as tf
from datetime import datetime
from tqdm import tqdm
import pykitti


In [9]:
batch_size = 1
sample_num = 1

In [10]:
indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))


In [27]:
indices_batch_indices.flags.writeable = False

In [21]:
np.array(indices_batch_indices[0][0])

array([0])

In [28]:
print(hash(indices_batch_indices ) )

TypeError: unhashable type: 'numpy.ndarray'

In [2]:
__predicate = "random-XYZ-grid-0.25"
_file_loc  = "/home/hasan/data/hdd8TB/KITTI/3D-object-detection/"
_file_loc_raw_data  = _file_loc + "dataset/"

__dataset = _file_loc+"/pointcnn-dataset/"+__predicate+"/val_files.txt"

__dataset_final_result = _file_loc+"/pointcnn-dataset/"+__predicate+"/val/"

__model = _file_loc+"/pointcnn-model/"+__predicate+"/pointcnn_seg_kitti3d_x8_2048_fps_2019-04-09-19-00-45_10534/ckpt-best/0.9259112-iter--150000"

pred_path = _file_loc+"/pointcnn-dataset/"+__predicate+"/val"


In [3]:

_traindata = _file_loc_raw_data + "training/val.txt"
_traindatacontent = open(_traindata, "r")
testlists = _traindatacontent.readlines()
testlists = [int(_d) for _d in testlists]

In [4]:
args = {
        'load_ckpt' : __model,
        'repeat_num' : 1,
        'setting' : "kitti3d_x8_2048_fps",
        'save_ply' : False,
        'model' : "pointcnn_seg",
        'max_point_num' : 10000,
        'filelist' : __dataset
}

model = importlib.import_module(args['model'])
setting_path = os.path.join(_file_loc+"/PointCNN/", args['model'])
sys.path.append(setting_path)
setting = importlib.import_module(args['setting'])

sample_num = setting.sample_num
max_point_num = args['max_point_num']
batch_size = args['repeat_num'] * int(math.ceil(max_point_num / sample_num))

######################################################################
# Placeholders
indices = tf.placeholder(tf.int32, shape=(batch_size, None, 2), name="indices")
is_training = tf.placeholder(tf.bool, name='is_training')
pts_fts = tf.placeholder(tf.float32, shape=(batch_size, max_point_num, setting.data_dim), name='points')
######################################################################

######################################################################
pts_fts_sampled = tf.gather_nd(pts_fts, indices=indices, name='pts_fts_sampled')
if setting.data_dim > 3:
    points_sampled, features_sampled = tf.split(pts_fts_sampled,
                                                [3, setting.data_dim - 3],
                                                axis=-1,
                                                name='split_points_features')
    if not setting.use_extra_features:
        features_sampled = None
else:
    points_sampled = pts_fts_sampled
    features_sampled = None

net = model.Net(points_sampled, features_sampled, is_training, setting)
seg_probs_op = tf.nn.softmax(net.logits, name='seg_probs')

# for restore model
saver = tf.train.Saver()

parameter_num = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])
print('{}-Parameter number: {:d}.'.format(datetime.now(), parameter_num))


with tf.Session() as sess:
    # Load the model
    saver.restore(sess, args['load_ckpt'])
    print('{}-Checkpoint loaded from {}!'.format(datetime.now(), args['load_ckpt']))

    indices_batch_indices = np.tile(np.reshape(np.arange(batch_size), (batch_size, 1, 1)), (1, sample_num, 1))

    folder = os.path.dirname(args['filelist'])
    filenames = [os.path.join(folder, line.strip()) for line in open(args['filelist'])]
    for filename in filenames:
        print('{}-Reading {}...'.format(datetime.now(), filename))
        data_h5 = h5py.File(filename)
        data = data_h5['data'][:,:,:4].astype(np.float32)
        data_num = data_h5['data_num'][...].astype(np.int32)
        original_label = data_h5['label'][...].astype(np.int64)
        batch_num = data.shape[0]

        labels_pred = np.full((batch_num, max_point_num), -1, dtype=np.int32)
        confidences_pred = np.zeros((batch_num, max_point_num), dtype=np.float32)

        #print('{}-{:d} testing batches.'.format(datetime.now(), batch_num))
        for batch_idx in tqdm(range(batch_num)):
            #if batch_idx % 10 == 0:
            #print('{}-Processing {} of {} batches.'.format(datetime.now(), batch_idx, batch_num))
            points_batch = data[[batch_idx] * batch_size, ...]
            point_num = data_num[batch_idx]
            if(point_num <= 10):
                continue 

            tile_num = int(math.ceil((sample_num * batch_size) / point_num))
            indices_shuffle = np.tile(np.arange(point_num), tile_num)[0:sample_num * batch_size]
            np.random.shuffle(indices_shuffle)
            indices_batch_shuffle = np.reshape(indices_shuffle, (batch_size, sample_num, 1))
            indices_batch = np.concatenate((indices_batch_indices, indices_batch_shuffle), axis=2)

            seg_probs = sess.run([seg_probs_op],
                                    feed_dict={
                                        pts_fts: points_batch,
                                        indices: indices_batch,
                                        is_training: False,
                                    })
            probs_2d = np.reshape(seg_probs, (sample_num * batch_size, -1))

            predictions = [(-1, 0.0)] * point_num
            for idx in range(sample_num * batch_size):
                point_idx = indices_shuffle[idx]
                probs = probs_2d[idx, :]
                confidence = np.amax(probs)
                label = np.argmax(probs)
                if confidence > predictions[point_idx][1]:
                    predictions[point_idx] = [label, confidence]
            labels_pred[batch_idx, 0:point_num] = np.array([label for label, _ in predictions])
            confidences_pred[batch_idx, 0:point_num] = np.array([confidence for _, confidence in predictions])

        filename_pred = filename[:-3] + __predicate+ '_pred.h5'
        print('{}-Saving {}...'.format(datetime.now(), filename_pred))
        file = h5py.File(filename_pred, 'w')
        file.create_dataset('data_num', data=data_num)
        file.create_dataset('label_seg', data=labels_pred)
        file.create_dataset('original_label', data=original_label)
        file.create_dataset('data', data=data)
        file.create_dataset('confidence', data=confidences_pred)
        has_indices = 'indices_split_to_full' in data_h5
        if has_indices:
            file.create_dataset('indices_split_to_full', data=data_h5['indices_split_to_full'][...])
        file.close()

        if args['save_ply']:
            print('{}-Saving ply of {}...'.format(datetime.now(), filename_pred))
            filepath_label_ply = os.path.join(filename_pred[:-3] + 'ply_label')
            data_utils.save_ply_property_batch(data[:, :, 0:3], labels_pred[...],
                                               filepath_label_ply, data_num[...], setting.num_class)
        ######################################################################
    print('{}-Done!'.format(datetime.now()))


Instructions for updating:
keep_dims is deprecated, use keepdims instead
2019-04-18 17:32:20.506435-Parameter number: 11510276.
INFO:tensorflow:Restoring parameters from /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-model/random-XYZ-grid-0.25/pointcnn_seg_kitti3d_x8_2048_fps_2019-04-09-19-00-45_10534/ckpt-best/0.9259112-iter--150000
2019-04-18 17:32:21.937763-Checkpoint loaded from /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-model/random-XYZ-grid-0.25/pointcnn_seg_kitti3d_x8_2048_fps_2019-04-09-19-00-45_10534/ckpt-best/0.9259112-iter--150000!
2019-04-18 17:32:21.938568-Reading /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_0.h5...


100%|██████████| 10240/10240 [42:51<00:00,  4.08it/s]


2019-04-18 18:15:18.898252-Saving /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_0random-XYZ-grid-0.25_pred.h5...


100%|██████████| 10240/10240 [42:23<00:00,  4.16it/s]


2019-04-18 18:16:03.246411-Reading /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_1.h5...
2019-04-18 18:58:53.250622-Saving /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_1random-XYZ-grid-0.25_pred.h5...
2019-04-18 18:59:26.964478-Reading /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_2.h5...


100%|██████████| 10030/10030 [41:25<00:00,  4.08it/s]


2019-04-18 19:41:18.004399-Saving /home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/./val/val_zero_2random-XYZ-grid-0.25_pred.h5...
2019-04-18 19:41:46.150189-Done!


In [5]:

pred_list = [pred for pred in os.listdir(pred_path) if pred.split(".")[-1] == "h5" and __predicate in pred]

In [6]:
pred_list

_result_preds = {}

for i in testlists:
    _result_preds[i] = [[]] #[[]Zero, []Half, FrameID]
    
for _file in tqdm(pred_list):

    
    origin_filename =  _file.replace(__predicate+ '_pred.h5', ".h5")
    #print(os.path.join(pred_path, origin_filename))
    input_data = h5py.File(os.path.join(pred_path, origin_filename))
    label = input_data['label'][...].astype(np.int32)
    
    frameids = np.unique(label)
    
    data = h5py.File(os.path.join(pred_path, _file))
    

    data_points = data['data'][:,:,:4].astype(np.float32)
    original_label = data['original_label'][...].astype(np.int64)

        
    labels_seg = data['label_seg'][...].astype(np.int64)
    indices = data['indices_split_to_full'][...].astype(np.int64)
    confidence = data['confidence'][...].astype(np.float32)
    data_num = data['data_num'][...].astype(np.int64)
    
    for frame_id in (frameids):
        labels_seg_frame = []
        confidence_frame = []
        idx_frame = []
        ptnum_frame = []
        indices_frame = np.nonzero(label==frame_id)
        for i in  indices_frame[0]:

            ptnum_frame.append(data_num[i])
            idx_frame.append(indices[i,:data_num[i]])
            labels_seg_frame.append(labels_seg[i,:data_num[i]])
            confidence_frame.append(confidence[i,:data_num[i]])
            
        idx_frame = np.concatenate(idx_frame)
        labels_seg_frame = np.concatenate(labels_seg_frame)
        labels_seg_frame = labels_seg_frame.astype(int)
        confidence_frame = np.concatenate(confidence_frame)
        
        is_half = 1 if "_half_" in origin_filename else 0
        
        _result_preds[frame_id][is_half] = [labels_seg_frame, confidence_frame, ptnum_frame, idx_frame]
        
    

100%|██████████| 3/3 [00:16<00:00,  5.59s/it]


In [7]:

final_label = {}

for _i in tqdm(testlists):
    
    if len(_result_preds[_i][0][3]) == 0: # or len(_result_preds[_i][1][3]) == 0 :
        print("NO DATA", _i)
        continue
        
    label_length = max(_result_preds[_i][0][3])+1 # max(max(_result_preds[_i][0][3]), max(_result_preds[_i][1][3]) )+1

    merged_label_zero = np.zeros((label_length),dtype=int)
    merged_confidence_zero = np.zeros((label_length),dtype=float)
    merged_label_half = np.zeros((label_length), dtype=int)
    merged_confidence_half = np.zeros((label_length), dtype=float)


    final_label[_i] =  np.zeros((label_length), dtype=int) #[[]Zero, []Half, FrameID]
    
    is_zero = True 
    for pred_data in _result_preds[_i]:
    
        labels_seg = pred_data[0].astype(np.int64)
        indices = pred_data[3].astype(np.int64)
        confidence = pred_data[1].astype(np.float32)
        data_num = np.array(pred_data[2]).astype(np.int64)

        if is_zero:
            for i in range(labels_seg.shape[0]):
                
                 if confidence[i] > merged_confidence_zero[indices[i]]:
                    merged_confidence_zero[indices[i]] = confidence[i]
                    merged_label_zero[indices[i]] = labels_seg[i]
                                
        else:
            for i in range(labels_seg.shape[0]):
                 if confidence[i] > merged_confidence_half[indices[i]]:
                    merged_confidence_half[indices[i]] = confidence[i]
                    merged_label_half[indices[i]] = labels_seg[i]
                
        is_zero = False

    final_label[_i][merged_confidence_zero >= merged_confidence_half] = merged_label_zero[merged_confidence_zero >= merged_confidence_half]
    final_label[_i][merged_confidence_zero < merged_confidence_half] = merged_label_half[merged_confidence_zero < merged_confidence_half]

100%|██████████| 2336/2336 [20:48<00:00,  1.78it/s]


In [8]:
np.savez(__dataset_final_result + __predicate +"-final-pred.npz", final_label)

In [9]:
__dataset_final_result + __predicate +"-final-pred.npz"

'/home/hasan/data/hdd8TB/KITTI/3D-object-detection//pointcnn-dataset/random-XYZ-grid-0.25/val/random-XYZ-grid-0.25-final-pred.npz'